# Подключение к БД и выполнение запроса

In [7]:
import psycopg2
from psycopg2 import OperationalError

In [8]:
conn = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="mhSrgTm85",
        host="localhost",
        port="5432"
    )
    
cursor = conn.cursor()

query = """
SELECT
    c.customer_id,
    CONCAT(c.first_name, ' ', c.last_name) AS customer_name,
    SUM(p.amount) AS total_payments,
    COUNT(r.rental_id) AS rental_count,
    CASE
        WHEN SUM(p.amount) >= 200 THEN 'Platinum'
        WHEN SUM(p.amount) >= 100 THEN 'Gold'
        WHEN SUM(p.amount) >= 50 THEN 'Silver'
        ELSE 'Ordinary'
    END AS customer_category,
    RANK() OVER (ORDER BY SUM(p.amount) DESC) AS payment_rank
FROM customer c
-- Первый JOIN: связываем customer с rental через customer_id
JOIN rental r ON c.customer_id = r.customer_id
-- Второй JOIN: связываем rental с payment через rental_id
JOIN payment p ON r.rental_id = p.rental_id
GROUP BY c.customer_id
ORDER BY total_payments DESC;
"""
# Execute the query
cursor.execute(query)

# Сохранение результатов

In [9]:
# Получаем столбцы результата (названия колонок)
columns = [desc[0] for desc in cursor.description]
    
# Создаем DataFrame из результатов
df = pd.DataFrame(cursor.fetchall(), columns=columns)
    
# Закрываем соединение
cursor.close()
conn.close()
    
# Сохраняем DataFrame в CSV-файл
df.to_excel('output.xlsx', index=False)
df

,customer_id,customer_name,total_payments,rental_count,customer_category,payment_rank
0,148,Eleanor Hunt,211.55,45,Platinum,1
1,526,Karl Seal,208.58,42,Platinum,2
2,178,Marion Snyder,194.61,39,Gold,3
3,137,Rhonda Kennedy,191.62,38,Gold,4
4,144,Clara Shaw,189.60,40,Gold,5
...,...,...,...,...,...,...
594,110,Tiffany Jordan,49.88,12,Ordinary,595
595,320,Anthony Schwab,47.85,15,Ordinary,596
596,248,Caroline Bowman,37.87,13,Ordinary,597
597,281,Leona Obrien,32.90,10,Ordinary,598
